# TP2 - Correcteur automatique de texte

## Etape 1 : Installation de SRI-LM

L'objectif de ce TP est de concevoir un correcteur automatique de tweets en utilisant un modèle de langage. Il vise à acquérir des compétences pratiques dans l'utilisation de ce type de modèle, en développant un correcteur automatique pour des textes courts, souvent mal écrits.

Télécharger et installer SRI-LM si ce n’est pas déjà fait. Vérifier que le logiciel est bien installé en exécutant une commande de test comme :




In [62]:
print("test")

test


In [63]:
# export SRILM=$PWD && make
# export SRILM=$PWD && make MACHINE_TYPE=i686-ubuntu
#echo 'export PATH=$PATH:$SRILM/bin/i686-ubuntu' >> ~/.bashrc


In [64]:
#source ~/.bashrc
#conda activate env_Tp2_tlp
#ngram-count -help

## Etape 2 : Génération d'un modèle de langage avec SRI-LM

Récupérer le corpus d'entraînement.

### Nettoyage du corpus
Créez une fonction qui nettoie le corpus en convertissant tout le texte en minuscules, en supprimant les ponctuations, et en ajoutant un espace après l'apostrophe ainsi que tout autre caractère spécial que vous jugerez non pertinent.

### Création du vocabulaire
Créer un fichier : *vocab.txt* qui contiendra les 10 000 mots les plus fréquents dans votre corpus.

### Questions
Expliquer brièvement les arguments :
* vocab
* text
* order
* lm


In [65]:
# ngram-count -vocab vocab.txt -text output.txt -order 3 -lm model.lm

## Nettoyage du text

In [1]:
import string
import re 

In [2]:
def sanitize_text(text):
    text_result=[]
    with open(text, "r") as f:
     for line in f:
        line=str.lower(line)
        line=re.sub(r"'"," ' ",line)
        line=re.sub(r"’"," ’ ",line)
        line = re.sub(r'[^\w\sàâäéèêëîïôöùûüç]', '', line)        
        text_result.append(line)
    return ' '.join(text_result)

In [68]:
output=sanitize_text("train.txt")

In [69]:
with open("output.txt","w") as fichier:
    fichier.write(str(output))

## Création du vocabulaire

In [70]:
def compteur_mot(text):
    frequence_mots={}
    mots = text.split()

    for i in mots:
        if i in frequence_mots:
            frequence_mots[i]+=1
        else:
            frequence_mots[i]=1
    
    return frequence_mots


In [45]:
compteur_mot(output)

{'de': 69600,
 'nouvelles': 226,
 'recherches': 120,
 'par': 8062,
 'hélicoptère': 62,
 'devaient': 70,
 'débuter': 14,
 'vendredi': 1049,
 'matin': 823,
 'afin': 309,
 'retrouver': 166,
 'les': 23099,
 'trois': 1305,
 'alpinistes': 23,
 'italiens': 24,
 'échoués': 1,
 'depuis': 1911,
 'dimanche': 1027,
 'dans': 12071,
 'le': 32881,
 'massif': 30,
 'des': 19965,
 'écrins': 3,
 'aton': 371,
 'appris': 323,
 'auprès': 385,
 'la': 39432,
 'gendarmerie': 215,
 'hautesalpes': 7,
 'refuge': 30,
 'temple': 10,
 'est': 12840,
 'priorité': 75,
 'a': 21099,
 'indiqué': 830,
 'colonel': 25,
 'robin': 10,
 'joubert': 1,
 'commandant': 58,
 'du': 15242,
 'groupement': 20,
 'ce': 6247,
 'situé': 120,
 'à': 26532,
 '2410': 2,
 'mètres': 165,
 'd': 20096,
 'altitude': 21,
 'n': 4890,
 'avait': 3363,
 'pu': 351,
 'être': 2398,
 'inspecté': 3,
 'jeudi': 1041,
 'fait': 2341,
 'une': 14430,
 'couche': 14,
 'nuageuse': 1,
 'trop': 299,
 'épaisse': 7,
 'or': 166,
 'selon': 3089,
 'géolocalisations': 1,
 'té

In [46]:
def creation_vocabulaire(text,max=10000):
    text=sanitize_text(text)
    frequence=compteur_mot(text)
    mots_tries = sorted(frequence.items(), key=lambda x: x[1], reverse=True)
    vocabulaire = [mot for mot, freq in mots_tries[:max]]

    with open('vocab.txt', 'w') as fichier_vocab:
        for mot in vocabulaire:
            fichier_vocab.write(mot + '\n')


creation_vocabulaire("output.txt")

vocab : fichier contenant le vocabulaire (vocab.txt).

text : le corpus d’entraînement.

order : ordre du modèle (bigramme, trigramme, etc.).

lm : fichier de sortie pour enregistrer le modèle de langage généré.

## Etape 3 : Implémentation du correcteur automatique de phrase

### Principe du correcteur
Le correcteur proposera des suggestions basées sur le modèle de langage n-gramme entraîné précédemment. Lorsque des erreurs ou des combinaisons peu probables sont détectées, le modèle proposera une alternative plus probable.

### Calculez la perplexité d'une phrase
La perplexité d'une phrase (ou d'un ensemble de phrases) se calcule ainsi :

In [ ]:
ngram -lm model.lm -ppl test.txt -order 3 -debug 2

In [48]:
test=sanitize_text("test_modify.txt")
with open("test.txt","w") as fichier:
    fichier.write(str(test))

### Écrire un script Python pour la correction de tweets

Le script prend un tweet en entrée et calcule la perplexité en utilisant le modèle de langage, puis parse le fichier de perplexité pour obtenir un score pour chaque mot. L'objectif est de proposer des alternatives pour les mots ayant une perplexité élevée, en suggérant des mots similaires mais plus probables.


### Étapes détaillées

#### Prise en entrée d'une phrase
Le script commence par recevoir une phrase.

#### Nettoyage du corpus
Appliquez les mêmes nettoyage sur la phrase que le corpus d'apprentissage.

#### Calcul de la perplexité
Utilisez SRI-LM pour calculer la perplexité de la phrase. Cela permet de mesurer à quel point une séquence est "probable" selon le modèle de langage.

#### Parsing des résultats
Parsez la sortie donné dans l'étape précédente pour obtenir un score par mot. Plus la perplexité est élevée, moins la séquence est probable.

#### Proposition d'alternatives (Distance de Levenshtein) :
* Identifiez les mots avec une perplexité élevée
* Utilisez la distance de Levenshtein pour proposer des alternatives plus proches des mots originales en termes de similarité sémantique (distance de levenshtein), mais potentiellement plus probables selon le modèle de langage.

#### Recalcul de la perplexité
* Après avoir généré des alternatives, recalculer la perplexité des nouvelles séquences pour identifier celle qui a la perplexité la plus basse.

#### Retourner le tweet corrigé
* Une fois la séquence la plus probable identifiée, générez le tweet corrigé avec les alternatives les plus appropriées.

In [3]:
import subprocess

def calculer_perplexite(tweet):
    with open("temp_tweet.txt", 'w') as f:
        f.write(tweet)
    cmd = ['bin/i686-ubuntu/ngram', '-lm', 'model.lm', '-ppl', 'temp_tweet.txt', '-order', '3', '-debug', '2']
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    o, e = proc.communicate()

    print('Output: ' + o.decode('utf-8'))
    return  o.decode('utf-8')

In [ ]:
result=calculer_perplexite('les géolocalisations des téléphones portable des alpinistes')

Output: les géolocalisations des téléphones portable des alpinistes
	p( les | <s> ) 	= [2gram] 0.06057856 [ -1.217681 ]
	p( <unk> | les ...) 	= [OOV] 0 [ -inf ]
	p( des | <unk> ...) 	= [1gram] 0.01638627 [ -1.78552 ]
	p( téléphones | des ...) 	= [2gram] 0.0001786989 [ -3.747878 ]
	p( portable | téléphones ...) 	= [1gram] 1.477731e-05 [ -4.830405 ]
	p( des | portable ...) 	= [1gram] 0.009821643 [ -2.007816 ]
	p( alpinistes | des ...) 	= [2gram] 0.0001786989 [ -3.747878 ]
	p( </s> | alpinistes ...) 	= [1gram] 0.008782532 [ -2.05638 ]
1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103

file temp_tweet.txt: 1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103



In [31]:
import math

def parse(output_text):
    lines = output_text.split('\n')
    pattern = re.compile(r'p\(\s*(\S+)\s*\|\s*.*\)\s*=\s*\[\S+\]\s*([\d.]+)\s*\[\s*([-\d.]+)\s*\]')
    results = []
    for line in lines:
        match = pattern.search(line)
        if match:
            word = match.group(1)              
            prob = float(match.group(2))        
            log_prob = float(match.group(3))    
            perplexity = math.exp(-log_prob)
            
            results.append({
                'word': word,
                'probability': prob,
                'log_probability': log_prob,
                'perplexity': perplexity
                
            })
    
    return results

In [32]:
tweet = 'les géolocalisations des téléphones portable des alpinistes'
output = calculer_perplexite(tweet)
results = parse(output)

Output: les géolocalisations des téléphones portable des alpinistes
	p( les | <s> ) 	= [2gram] 0.06057856 [ -1.217681 ]
	p( <unk> | les ...) 	= [OOV] 0 [ -inf ]
	p( des | <unk> ...) 	= [1gram] 0.01638627 [ -1.78552 ]
	p( téléphones | des ...) 	= [2gram] 0.0001786989 [ -3.747878 ]
	p( portable | téléphones ...) 	= [1gram] 1.477731e-05 [ -4.830405 ]
	p( des | portable ...) 	= [1gram] 0.009821643 [ -2.007816 ]
	p( alpinistes | des ...) 	= [2gram] 0.0001786989 [ -3.747878 ]
	p( </s> | alpinistes ...) 	= [1gram] 0.008782532 [ -2.05638 ]
1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103

file temp_tweet.txt: 1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103



In [33]:
print(f"le resultat {results}")

le resultat [{'word': 'les', 'probability': 0.06057856, 'log_probability': -1.217681, 'perplexity': 3.379341945973908}, {'word': 'des', 'probability': 0.01638627, 'log_probability': -1.78552, 'perplexity': 5.962679735030218}, {'word': 'téléphones', 'probability': 0.0001786989, 'log_probability': -3.747878, 'perplexity': 42.43094793012879}, {'word': 'des', 'probability': 0.009821643, 'log_probability': -2.007816, 'perplexity': 7.447035248756219}, {'word': 'alpinistes', 'probability': 0.0001786989, 'log_probability': -3.747878, 'perplexity': 42.43094793012879}, {'word': '</s>', 'probability': 0.008782532, 'log_probability': -2.05638, 'perplexity': 7.817618745991827}]


In [34]:
def mots_a_corriger(results, seuil_perplexite):
    # Filtrer les mots dont la perplexité dépasse le seuil
    mots_corriges = [mot['word'] for mot in results if mot['perplexity'] > seuil_perplexite]
    return mots_corriges

In [35]:
mots_probables_a_corriger = mots_a_corriger(results, seuil_perplexite=30)
mots_probables_a_corriger


['téléphones', 'alpinistes']

In [ ]:
import Levenshtein
def charger_vocabulaire(fichier_vocab):
    # Lire le fichier vocab.txt et charger les mots dans une liste
    with open(fichier_vocab, 'r') as f:
        vocabulaire = [ligne.strip() for ligne in f] 
    return vocabulaire

def trouver_alternatives(mot, vocabulaire, seuil=2):
    # Chercher des mots proches dans le vocabulaire entier
    alternatives = []
    for mot_voc in vocabulaire:
        if Levenshtein.distance(mot, mot_voc) <= seuil:  
            alternatives.append(mot_voc)
    return alternatives

vocabulaire = charger_vocabulaire('vocab.txt')

In [36]:
vocabulaire

['de',
 'la',
 'le',
 'l',
 'à',
 'les',
 'a',
 'et',
 'd',
 'des',
 'en',
 'un',
 'du',
 'une',
 'est',
 'pour',
 'dans',
 'qui',
 'il',
 'que',
 'sur',
 'au',
 'par',
 'pas',
 'été',
 'ce',
 'ont',
 'son',
 'qu',
 's',
 'n',
 'plus',
 'se',
 'ne',
 'avec',
 'sont',
 'mais',
 'avait',
 'c',
 'cette',
 'aux',
 'sa',
 'selon',
 'deux',
 'ans',
 'françois',
 'elle',
 'président',
 'ministre',
 'après',
 'être',
 'y',
 'je',
 'ses',
 'fait',
 'on',
 'était',
 'nous',
 'leur',
 'ou',
 'comme',
 'france',
 'ils',
 'ump',
 'ces',
 'depuis',
 'avoir',
 'contre',
 'lui',
 'hollande',
 'aussi',
 'même',
 'premier',
 'entre',
 'tout',
 'faire',
 'gouvernement',
 'français',
 'personnes',
 'très',
 'déclaré',
 'si',
 'où',
 'dont',
 'sarkozy',
 'alors',
 'avant',
 'pays',
 'trois',
 'lors',
 'nicolas',
 'atil',
 'sans',
 'mardi',
 'mercredi',
 'parti',
 'police',
 'euros',
 'encore',
 'tous',
 'politique',
 'également',
 'homme',
 'autres',
 'lundi',
 'etat',
 'plusieurs',
 'vendredi',
 'jeudi',


In [ ]:
def corriger_tweet(tweet, vocabulaire, seuil_perplexite=30, seuil_levenshtein=2):
    # 1. Calculer la perplexité du tweet
    output = calculer_perplexite(tweet)
    results = parse(output)
    
    # 2. Identifier les mots avec une perplexité élevée
    mots_probables_a_corriger = mots_a_corriger(results, seuil_perplexite)
    print(f'mot à corriger: {mots_probables_a_corriger}')
    
    # 3. Corriger les mots en proposant des alternatives
    mots_tweet = tweet.split()  # Diviser le tweet en mots
    for i, mot in enumerate(mots_tweet):
        if mot in mots_probables_a_corriger:
            alternatives = trouver_alternatives(mot, vocabulaire, seuil_levenshtein)
            print(f"Les alternatives pour '{mot}' sont: {alternatives}")
            if alternatives:
                for alt in alternatives:
                    if alt != mot:  
                        mots_tweet[i] = alt
                        break  
    
    # 4. Reconstituer le tweet corrigé
    tweet_corrige = ' '.join(mots_tweet)
    
    return tweet_corrige


In [25]:
vocabulaire = charger_vocabulaire('vocab.txt')
tweet = ' les géolocalisations des téléphones portable des alpinistes'

tweet_corrige = corriger_tweet(tweet, vocabulaire)
print("Tweet corrigé :", tweet_corrige)

Output: les géolocalisations des téléphones portable des alpinistes
	p( les | <s> ) 	= [2gram] 0.06057856 [ -1.217681 ]
	p( <unk> | les ...) 	= [OOV] 0 [ -inf ]
	p( des | <unk> ...) 	= [1gram] 0.01638627 [ -1.78552 ]
	p( téléphones | des ...) 	= [2gram] 0.0001786989 [ -3.747878 ]
	p( portable | téléphones ...) 	= [1gram] 1.477731e-05 [ -4.830405 ]
	p( des | portable ...) 	= [1gram] 0.009821643 [ -2.007816 ]
	p( alpinistes | des ...) 	= [2gram] 0.0001786989 [ -3.747878 ]
	p( </s> | alpinistes ...) 	= [1gram] 0.008782532 [ -2.05638 ]
1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103

file temp_tweet.txt: 1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103

mot à corriger: ['téléphones', 'alpinistes']
Les alternatives pour 'téléphones' sont: ['téléphone', 'téléphones', 'téléphoné']
Les alternatives pour 'alpinistes' sont: ['alpinistes']
Tweet corrigé : les géolocalisations des téléphone portable des alpinistes


In [26]:
calculer_perplexite('les géolocalisations des téléphones portable des alpinistes')

Output: les géolocalisations des téléphones portable des alpinistes
	p( les | <s> ) 	= [2gram] 0.06057856 [ -1.217681 ]
	p( <unk> | les ...) 	= [OOV] 0 [ -inf ]
	p( des | <unk> ...) 	= [1gram] 0.01638627 [ -1.78552 ]
	p( téléphones | des ...) 	= [2gram] 0.0001786989 [ -3.747878 ]
	p( portable | téléphones ...) 	= [1gram] 1.477731e-05 [ -4.830405 ]
	p( des | portable ...) 	= [1gram] 0.009821643 [ -2.007816 ]
	p( alpinistes | des ...) 	= [2gram] 0.0001786989 [ -3.747878 ]
	p( </s> | alpinistes ...) 	= [1gram] 0.008782532 [ -2.05638 ]
1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103

file temp_tweet.txt: 1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103



'les géolocalisations des téléphones portable des alpinistes\n\tp( les | <s> ) \t= [2gram] 0.06057856 [ -1.217681 ]\n\tp( <unk> | les ...) \t= [OOV] 0 [ -inf ]\n\tp( des | <unk> ...) \t= [1gram] 0.01638627 [ -1.78552 ]\n\tp( téléphones | des ...) \t= [2gram] 0.0001786989 [ -3.747878 ]\n\tp( portable | téléphones ...) \t= [1gram] 1.477731e-05 [ -4.830405 ]\n\tp( des | portable ...) \t= [1gram] 0.009821643 [ -2.007816 ]\n\tp( alpinistes | des ...) \t= [2gram] 0.0001786989 [ -3.747878 ]\n\tp( </s> | alpinistes ...) \t= [1gram] 0.008782532 [ -2.05638 ]\n1 sentences, 7 words, 1 OOVs\n0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103\n\nfile temp_tweet.txt: 1 sentences, 7 words, 1 OOVs\n0 zeroprobs, logprob= -19.39356 ppl= 589.5332 ppl1= 1707.103\n'

In [27]:
calculer_perplexite("les géolocalisations des téléphone portable des alpinistes")

Output: les géolocalisations des téléphone portable des alpinistes
	p( les | <s> ) 	= [2gram] 0.06057856 [ -1.217681 ]
	p( <unk> | les ...) 	= [OOV] 0 [ -inf ]
	p( des | <unk> ...) 	= [1gram] 0.01638627 [ -1.78552 ]
	p( téléphone | des ...) 	= [1gram] 6.460874e-06 [ -5.189709 ]
	p( portable | téléphone ...) 	= [2gram] 0.1825397 [ -0.7386427 ]
	p( des | portable ...) 	= [1gram] 0.01025789 [ -1.988942 ]
	p( alpinistes | des ...) 	= [2gram] 0.0001786989 [ -3.747878 ]
	p( </s> | alpinistes ...) 	= [1gram] 0.008782532 [ -2.05638 ]
1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -16.72475 ppl= 245.0475 ppl1= 612.9976

file temp_tweet.txt: 1 sentences, 7 words, 1 OOVs
0 zeroprobs, logprob= -16.72475 ppl= 245.0475 ppl1= 612.9976



'les géolocalisations des téléphone portable des alpinistes\n\tp( les | <s> ) \t= [2gram] 0.06057856 [ -1.217681 ]\n\tp( <unk> | les ...) \t= [OOV] 0 [ -inf ]\n\tp( des | <unk> ...) \t= [1gram] 0.01638627 [ -1.78552 ]\n\tp( téléphone | des ...) \t= [1gram] 6.460874e-06 [ -5.189709 ]\n\tp( portable | téléphone ...) \t= [2gram] 0.1825397 [ -0.7386427 ]\n\tp( des | portable ...) \t= [1gram] 0.01025789 [ -1.988942 ]\n\tp( alpinistes | des ...) \t= [2gram] 0.0001786989 [ -3.747878 ]\n\tp( </s> | alpinistes ...) \t= [1gram] 0.008782532 [ -2.05638 ]\n1 sentences, 7 words, 1 OOVs\n0 zeroprobs, logprob= -16.72475 ppl= 245.0475 ppl1= 612.9976\n\nfile temp_tweet.txt: 1 sentences, 7 words, 1 OOVs\n0 zeroprobs, logprob= -16.72475 ppl= 245.0475 ppl1= 612.9976\n'

In [28]:
import Levenshtein

mot1 = "matheus"
mot2 = "mateus"

distance = Levenshtein.distance(mot1, mot2)
print("Distance de Levenshtein :", distance) 

Distance de Levenshtein : 1


In [37]:
trouver_alternatives(mot="parlement",vocabulaire=vocabulaire)

['parlement',
 'largement',
 'parlent',
 'rarement',
 'purement',
 'armement',
 'paiement']

In [38]:
trouver_alternatives(mot="fournir",vocabulaire=vocabulaire)

['fournir',
 'tourner',
 'mourir',
 'fourni',
 'courir',
 'nourrir',
 'fournis',
 'fournies']